# TrueSkill Model
Implementing a variant of the [TrueSkill](http://papers.nips.cc/paper/3079-trueskilltm-a-bayesian-skill-rating-system.pdf) model, a player ranking system for competitive games originally developed for Halo 2. It is a generalization of the Elo rating system in Chess.

In [ ]:
!pip install wget
import os
import os.path

import matplotlib.pyplot as plt
import wget

import pandas as pd


import numpy as np
from scipy.stats import norm
import scipy.io
import scipy.stats
import torch
import random
from torch.distributions.normal import Normal

from functools import partial

import matplotlib.pyplot as plt


##1. Examining the posterior for only two players and toy data

In [ ]:
#computes the log of the prior, jointly evaluated over all player's skills.
def log_joint_prior(zs_array):
  log_prob = torch.sum(Normal(0, 1).log_prob(zs_array))
  return log_prob

#evaluates the log-likelihood that player with skill z_a beat player with skill z_b
def logp_a_beats_b(z_a, z_b):
  logp_a = -torch.logaddexp(torch.tensor(0.0), (z_b-z_a))
  return logp_a

def logp_b_beats_a(z_a, z_b):
  logp_b = -torch.logaddexp(torch.tensor(0.0), (z_a-z_b))
  return logp_b

def plot_isocontours(ax, func, steps=100):
    x = torch.linspace(-4, 4, steps=steps)
    y = torch.linspace(-4, 4, steps=steps)
    X, Y = torch.meshgrid(x, y, indexing="ij")
    Z = func(X, Y)
    cs = plt.contour(X, Y, Z )
    plt.clabel(cs, inline=1, fontsize=10)
    ax.set_yticks([])
    ax.set_xticks([])

def plot_2d_fun(f, x_axis_label="", y_axis_label="", scatter_pts=None):
    fig = plt.figure(figsize=(8,8), facecolor='white')
    ax = fig.add_subplot(111, frameon=False)
    ax.set_xlabel(x_axis_label)
    ax.set_ylabel(y_axis_label)
    plot_isocontours(ax, f)
    if scatter_pts is not None:
      plt.scatter(scatter_pts[:,0], scatter_pts[:, 1])
    plt.plot([4, -4], [4, -4], 'b--')   # Line of equal skill
    plt.show(block=True)
    plt.draw()

 # plot the isocontours of the joint prior over their skills
def log_prior_over_2_players(z1, z2):

  prior2 = Normal(0,1).log_prob(z1) + Normal(0,1).log_prob(z2)
  return prior2

def prior_over_2_players(z1, z2):
  return torch.exp(log_prior_over_2_players(z1, z2))

plot_2d_fun(prior_over_2_players, "Player A Skill", "Player B Skill")


def likelihood_over_2_players(z1, z2):
  return torch.exp(logp_a_beats_b(z1, z2))

plot_2d_fun(likelihood_over_2_players, "Player A Skill", "Player B Skill")

# Plot isocountours of the joint posterior over z_A and z_B given that player A beat player B in one match
def log_posterior_A_beat_B(z1, z2):
  logp = log_prior_over_2_players(z1, z2) + logp_a_beats_b(z1, z2)
  return logp

def posterior_A_beat_B(z1, z2):
  post = torch.exp(log_posterior_A_beat_B(z1, z2))
  return post

plot_2d_fun(posterior_A_beat_B, "Player A Skill", "Player B Skill")

#plot isocountours of the joint posterior over $z_A$ and $z_B$ given that 5 matches were played, and player A beat player B in all matches
def log_posterior_A_beat_B_5_times(z1, z2):
  return log_prior_over_2_players(z1,z2) + 5 * logp_a_beats_b(z1, z2)

def posterior_A_beat_B_5_times(z1, z2):
  return torch.exp(log_posterior_A_beat_B_5_times(z1, z2))

plot_2d_fun(posterior_A_beat_B_5_times, "Player A Skill", "Player B Skill")


# Plot isocontours of the joint posterior over z_A and z_B given that 10 matches were played, and each player beat the other 5 times.
def log_posterior_beat_each_other_5_times(z1, z2):
  return log_prior_over_2_players(z1,z2) + 5 * logp_a_beats_b(z1, z2) + 5 * logp_b_beats_a(z1, z2)

def posterior_beat_each_other_5_times(z1, z2):
  return torch.exp(log_posterior_beat_each_other_5_times(z1, z2))

plot_2d_fun(posterior_beat_each_other_5_times, "Player A Skill", "Player B Skill")



## 2. Approximate posterior distributions with gradient-based Hamiltonian Monte Carlo.


In [ ]:
random.seed(42)
torch.manual_seed(42)
# Hamiltonian Monte Carlo
from tqdm import trange, tqdm_notebook  # Progress meters

def leapfrog(params_t0, momentum_t0, stepsize, logprob_grad_fun):
  # Performs a reversible update of parameters and momentum
  # See https://en.wikipedia.org/wiki/Leapfrog_integration
  momentum_thalf = momentum_t0    + 0.5 * stepsize * logprob_grad_fun(params_t0)
  params_t1 =      params_t0      +       stepsize * momentum_thalf
  momentum_t1 =    momentum_thalf + 0.5 * stepsize * logprob_grad_fun(params_t1)
  return params_t1, momentum_t1

def iterate_leapfrogs(theta, v, stepsize, num_leapfrog_steps, grad_fun):
  for i in range(0, num_leapfrog_steps):
    theta, v = leapfrog(theta, v, stepsize, grad_fun)
  return theta, v

def metropolis_hastings(state1, state2, log_posterior):
  # Compares the log_posterior at two values of parameters,
  # and accepts the new values proportional to the ratio of the posterior
  # probabilities.
  accept_prob = torch.exp(log_posterior(state2) - log_posterior(state1))
  if random.random() < accept_prob:
    return state2  # Accept
  else:
    return state1  # Reject

def draw_samples(num_params, stepsize, num_leapfrog_steps, n_samples, log_posterior):
  theta = torch.zeros(num_params)

  def log_joint_density_over_params_and_momentum(state):
    params, momentum = state
    m = Normal(0., 1.)
    return m.log_prob(momentum).sum(axis=-1) + log_posterior(params)

  def grad_fun(zs):
    zs = zs.detach().clone()
    zs.requires_grad_(True)
    y = log_posterior(zs)
    y.backward()
    return zs.grad


  sampleslist = []
  for i in trange(0, n_samples):
    sampleslist.append(theta)

    momentum = torch.normal(0, 1, size = np.shape(theta))

    theta_new, momentum_new = iterate_leapfrogs(theta, momentum, stepsize, num_leapfrog_steps, grad_fun)

    theta, momentum = metropolis_hastings((theta, momentum), (theta_new, momentum_new), log_joint_density_over_params_and_momentum)
  return torch.stack((sampleslist))


# approximate the joint posterior where we observe player A winning 1 game.
num_players = 2
num_leapfrog_steps = 20
n_samples = 2500
stepsize = 0.01

def log_posterior_a(zs):
  z1, z2 = zs[0], zs[1]
  return log_posterior_A_beat_B(z1, z2)

samples_a = draw_samples(num_players, stepsize, num_leapfrog_steps, n_samples, log_posterior_a)
plot_2d_fun(posterior_A_beat_B, "Player A Skill", "Player B Skill", samples_a)

# approximate the joint posterior where we observe player A winning 5 games against player B
# Hyperparameters
num_players = 2
num_leapfrog_steps = 20
n_samples = 2500
stepsize = 0.01
key = 42


def log_posterior_b(zs):
  z1 = zs[0]
  z2 = zs[1]
  return log_posterior_A_beat_B_5_times(z1, z2)

samples_b = draw_samples(num_players, stepsize, num_leapfrog_steps, n_samples, log_posterior_b)

plot_2d_fun(posterior_A_beat_B_5_times, "Player A Skill", "Player B Skill", samples_b)

# approximate the joint posterior where we observe player A winning 5 games and player B winning 5 games.
num_players = 2
num_leapfrog_steps = 20
n_samples = 2500
stepsize = 0.01


def log_posterior_c(zs):
  z1 = zs[0]
  z2 = zs[1]
  return log_posterior_beat_each_other_5_times(z1,z2)

samples_c = draw_samples(num_players, stepsize, num_leapfrog_steps, n_samples, log_posterior_c)
plot_2d_fun(posterior_beat_each_other_5_times, "Player A Skill", "Player B Skill", samples_c)

# Stochastic Variational Inference in the TrueSkill Model


## Model definition

We assume that each player has a true, but unknown skill $z_i \in \mathbb{R}$.
We use $N$ to denote the number of players.

### The prior:
The prior over each player's skill is a standard normal distribution, and all player's skills are *a priori* independent.

### The likelihood:
For each observed game, the probability that player $A$ beats player $B$, given the player's skills $z_A$ and $z_B$, is:
$$p(A \,\, \text{beat} \,\, B | z_A, z_B) = \sigma(z_A - z_B)$$
where
$$\sigma(y) = \frac{1}{1 + \exp(-y)}$$
We chose this function simply because it's close to zero or one when the player's skills are very different, and equals one-half when the player skills are the same.  This likelihood function is the only thing that gives meaning to the latent skill variables $z_1$

In [ ]:
!pip install wget
import os
import os.path

import matplotlib.pyplot as plt
import wget

import pandas as pd


import numpy as np
from scipy.stats import norm
import scipy.io
import scipy.stats
import torch
import random
from torch import nn
from torch.distributions.normal import Normal

from functools import partial
from tqdm import trange, tqdm_notebook

import matplotlib.pyplot as plt

# Helper function
def diag_gaussian_log_density(x, mu, std):
    # axis=-1 means sum over the last dimension.
    m = Normal(mu, std)
    return torch.sum(m.log_prob(x), axis=-1)

## 1. Implementing the Model


In [ ]:
def log_joint_prior(zs_array):
    return diag_gaussian_log_density(zs_array, torch.tensor([0.0]), torch.tensor([1.0]))

def logp_a_beats_b(z_a, z_b):
    return -torch.logaddexp(torch.tensor([0.0]), z_b - z_a)

def log_prior_over_2_players(z1, z2):
    m = Normal(torch.tensor([0.0]), torch.tensor([[1.0]]))
    return m.log_prob(z1) + m.log_prob(z2)

def prior_over_2_players(z1, z2):
    return torch.exp(log_prior_over_2_players(z1, z2))

def log_posterior_A_beat_B(z1, z2):
    return log_prior_over_2_players(z1, z2) + logp_a_beats_b(z1, z2)

def posterior_A_beat_B(z1, z2):
    return torch.exp(log_posterior_A_beat_B(z1, z2))

def log_posterior_A_beat_B_10_times(z1, z2):
    return log_prior_over_2_players(z1, z2) + 10.0 * logp_a_beats_b(z1, z2)

def posterior_A_beat_B_10_times(z1, z2):
    return torch.exp(log_posterior_A_beat_B_10_times(z1, z2))

def log_posterior_beat_each_other_10_times(z1, z2):
    return log_prior_over_2_players(z1, z2) \
        + 10.* logp_a_beats_b(z1, z2) \
        + 10.* logp_a_beats_b(z2, z1)

def posterior_beat_each_other_10_times(z1, z2):
    return torch.exp(log_posterior_beat_each_other_10_times(z1, z2))


def plot_isocontours(ax, func, xlimits=[-4, 4], ylimits=[-4, 4], steps=101, cmap="summer"):
    x = torch.linspace(*xlimits, steps=steps)
    y = torch.linspace(*ylimits, steps=steps)
    X, Y = torch.meshgrid(x, y)
    Z = func(X, Y)
    plt.contour(X, Y, Z, cmap=cmap)
    ax.set_yticks([])
    ax.set_xticks([])

def plot_2d_fun(f, x_axis_label="", y_axis_label="", f2=None, scatter_pts=None):
    fig = plt.figure(figsize=(8,8), facecolor='white')
    ax = fig.add_subplot(111, frameon=False)
    ax.set_xlabel(x_axis_label)
    ax.set_ylabel(y_axis_label)
    plot_isocontours(ax, f)
    if f2 is not None:
      plot_isocontours(ax, f2, cmap='winter')

    if scatter_pts is not None:
      plt.scatter(scatter_pts[:,0], scatter_pts[:, 1])
    plt.plot([4, -4], [4, -4], 'b--')   # Line of equal skill
    plt.show(block=True)
    plt.draw()

## 2. Stochastic Variational Inference on Two Players and Toy Data
The original Trueskill paper from 2007 used message passing.
Here, I will approximate posterior distributions with gradient-based stochastic variational inference.

In [ ]:
def diag_gaussian_samples(mean, log_std, num_samples):
    epsilon = torch.randn(num_samples, mean.shape[0])
    samples = mean + torch.exp(log_std) * epsilon
    return samples

import math
def diag_gaussian_logpdf(x, mean, log_std):
    return torch.sum(Normal(mean, torch.exp(log_std)).log_prob(x), axis=-1)

def batch_elbo(logprob, mean, log_std, num_samples):
    samples = diag_gaussian_samples(mean, log_std, num_samples)
    log_variational_density = diag_gaussian_logpdf(samples, mean, log_std)
    log_density_samples = logprob(samples)
    elbo_estimate = torch.mean(log_density_samples - log_variational_density)

    return elbo_estimate

# Hyperparameters
num_players = 2
n_iters = 800
stepsize = 0.0001
num_samples_per_iter = 50

def log_posterior_A_beat_B_10_times_1_arg(z1z2):
  return log_posterior_A_beat_B_10_times(z1z2[:,0], z1z2[:,1])

def objective(params):  # The loss function to be minimized.
  mean, log_std = params
  return -batch_elbo(log_posterior_A_beat_B_10_times_1_arg, mean, log_std, num_samples_per_iter)

Initialize a set of variational parameters and optimize them (written for you already) to approximate the joint distribution where we observe player A winning 10 games. Report the final loss. Also plot the optimized variational approximation contours and the target distribution on the same axes.


In [ ]:
def callback(params, t):
  if t % 25 == 0:
    print("Iteration {} lower bound {}".format(t, objective(params)))

# Set up optimizer.
D = 2
init_log_std  = torch.tensor([0.0, 0.0], requires_grad=True) # TODO.
init_mean = torch.tensor([0.0, 0.0], requires_grad=True)# TODO

params = [init_mean, init_log_std]
optimizer = torch.optim.SGD(params, lr=stepsize, momentum=0.9)

def update():
    optimizer.zero_grad()
    loss = objective(params)
    loss.backward()
    optimizer.step()

# Main loop.
print("Optimizing variational parameters...")
for t in trange(0, n_iters):
    update()
    callback(params, t)


def approx_posterior_2d(z1, z2):

    mean, logstd = params[0].detach(), params[1].detach()
    zs = torch.stack([z1.flatten(), z2.flatten()], dim=1)
    probs = torch.exp(diag_gaussian_logpdf(zs, mean, logstd))
    return probs.view(z1.shape)

plot_2d_fun(posterior_A_beat_B_10_times, "Player A Skill", "Player B Skill",
            f2=approx_posterior_2d)


# Hyperparameters
n_iters = 100
stepsize = 0.0001
num_samples_per_iter = 50

def log_posterior_beat_each_other_10_times_1_arg(z1z2):
    return log_posterior_A_beat_B_10_times(z1z2[:,0], z1z2[:,1])

def objective(params):
    mean, log_std = params

    samples = diag_gaussian_samples(mean, log_std, num_samples_per_iter)
    log_probs = log_posterior_beat_each_other_10_times_1_arg(samples)

    return -torch.mean(log_probs)

# Main loop.
init_mean = torch.tensor([0.0, 0.0], requires_grad=True)
init_log_std = torch.tensor([0.0, 0.0], requires_grad=True)
params = (init_mean, init_log_std)
optimizer = torch.optim.SGD(params, lr=stepsize, momentum=0.9)

print("Optimizing variational parameters...")
for t in trange(0, n_iters):
    update()
    callback(params, t)

plot_2d_fun(posterior_beat_each_other_10_times, "Player A Skill", "Player B Skill",
            f2=approx_posterior_2d)